# Converting City Rawdata into Boundary GeoJSON

## Part 1: Introduction

This Jupyter Notebook is intended to create city boundary GeoJSON based on city GeoJSON/Shapefile.

## Part 2: Preparation

We will be using **Jupyter Notebook(anaconda 3)** to edit and run the script. Information on Anaconda installation can be found <a href='https://docs.anaconda.com/anaconda/install/'>here</a>. Please note that this script is running on Python 3.

***Usually, You can download city boundary data from state data portals.***

To run this script you need:
- city GeoJSON/Shapefile stored in **state** folder
- county GeoJSON/Shapefile stored in **state** folder
- directory path (**geojsons** folder > **state** folder)

The script currently prints one GeoJSON file:
- **state_City_boundaries.json**

>Original created on Feb 4 2021<br>
@author: Yijing Zhou @YijingZhou33

## Part 3: Get Started

###  Step 1: Import modules

In [ ]:
import pandas as pd
import os
import geopandas as gpd
import json
from itertools import chain
import string
import folium

### Step 2: Manual items to change
> Uncomment one of the code blocks based on file type

In [ ]:
###### Target state ######
state = 'Iowa'

###### Rawdata is Shapefile ######
citydata = 'City'
countydata = 'IL_BNDY_County_Py'

###### Rawdata is GeoJSON ######
# citydata = 'City'
# countydata = 'ODOT_County_Boundaries'

### Step 3: Set file path

In [ ]:
output = os.path.join('geojson', state, state + '_City_boundaries.json')

## Part 4: Build up city GeoJSON schema

###  Step 4: Convert both city and county rawdata into GeoJSON
> Uncomment one of the code blocks based on file type

In [ ]:
###### Rawdata is Shapefile ######
def shp_to_gdf(rawdata):
    path = os.path.join('geojson', state, rawdata)
    shp = gpd.read_file(path, driver = 'shapefile').to_crs('EPSG:4326')
    return shp

# **************** uncomment **********************
gdf_city = shp_to_gdf(citydata)
gdf_county = shp_to_gdf(countydata)
# *************************************************

###### Rawdata is GeoJSON ######
# def geojson_to_gdf(rawdata):
#     path = os.path.join('geojson', state, rawdata + '.geojson')
#     geojson = gpd.read_file(path).to_crs('EPSG:4326')
#     return geojson

# **************** uncomment **********************
# gdf_city = geojson_to_gdf(citydata)
# gdf_county = geojson_to_gdf(countydata)
# *************************************************

###  Step 5: Inspect the dataframe to find out the column of city name

We're going to extract the columns containing city name, county name and geometry information.

Here're some possible situations you may come across:
1. city dataframe includes county name
    - if city rawdata includes other administrative levels, we need to select by multiple attributes
2. city dataframe does not include county name
    - spatial join is required for city and county geodataframes
    
> You probably have to combine code based on what you've got. 

In [ ]:
gdf_city.head()

In [ ]:
gdf_county.head()

---

#### City rawdata includes other administrative levels

In [ ]:
# classname = 'SUBTYPE'
# citytype = 'C'

# gdf_city = gdf_city[gdf_city[classname] == citytype]

#### If spatial join is required -- city rawdata does not include county column
Also provide one column to uniquely identify each city

In [ ]:
# def group_by_city(identifier, cityname, countyname):
#     gdf_sjoin = gpd.sjoin(gdf_city, gdf_county, op = 'intersects', how='left')[[identifier, cityname, countyname]].astype(str)
#     # group records by city name
#     df_group = gdf_sjoin.groupby(cityname)[countyname].apply(list).reset_index(
#                 name = countyname)
#     gdf_merged = gdf_city.merge(df_group, on = cityname).rename(
#         columns = {cityname: 'City', countyname: 'County'})
#     gdf_merged['City'] = gdf_merged['City'].apply(lambda row: string.capwords(str(row)))
#     gdf_merged['County'] = gdf_merged['County'].apply(lambda row: [string.capwords(str(elem)) + ' County' for elem in row])
#     gdf_merged['County'] = gdf_merged['County'].apply(lambda row: ', '.join(list(set(row))))
#     return gdf_merged[['City', 'County', 'geometry']]

# gdf_merged = group_by_city(identifier, cityname, countyname)

#### Rename colmns

In [ ]:
# gdf_city = gdf_city.rename(columns = {'CITY_NAME': 'City', 'COUNTY_NAM': 'County'})[['City', 'County', 'geometry']].reset_index(drop = True)

#### Capitalize the first letter of each word in the county name

In [ ]:
# gdf_merged['City'] = gdf_merged['City'].apply(lambda row: string.capwords(str(row)))
# gdf_merged['County'] = gdf_merged['County'].apply(lambda row: string.capwords(str(row)) + ' County')

---

### Step 6: Convert GeoJSON into JSON

In [ ]:
def conversion(inputfile):
    ## convert file to json 
    inputfile = json.loads(inputfile.to_json())
    ## display features properties as dataframe
    df = pd.json_normalize(inputfile['features'])
    return df

df_merged = conversion(gdf_merged)

###  Step 7: Create bounding box

In [ ]:
def round_coordinates(l, precision):
    def round_element(e):
        if isinstance(e, list):
            return round_coordinates(e, precision)
        else:
            return round(e, precision)
    return [round_element(e) for e in l]

df_merged['geometry.coordinates'] = round_coordinates(df_merged['geometry.coordinates'], 4)
df_merged.head()

## Part 5: Create County GeoJSON

###  Step 8: Create geojson features

In [ ]:
def create_geojson_features(df):
    print('> Creating GeoJSON features...')
    features = []
    geojson = {
        'type': 'FeatureCollection',
        'features': features
    }
        
    for _, row in df.iterrows():
        if type(row['geometry.coordinates'][0][0][0]) is float:
            geometry_type = 'Polygon'
        else:
            geometry_type = 'MultiPolygon'
        feature = {
            'type': 'Feature',
            'geometry': {
                'type': geometry_type, 
                'coordinates': row['geometry.coordinates']
            },
            'properties': {
                'City': row['properties.City'], 
                'County': row['properties.County'], 
                'State': state
            }
        }

        features.append(feature)
    return geojson

data_geojson = create_geojson_features(df_merged)

###  Step 9: Generate geojson file

In [ ]:
with open(output, 'w') as txtfile:
    json.dump(data_geojson, txtfile)
print('> Creating GeoJSON file...')

## Part 6: Inspect bounding box map

In [ ]:
print('> Making map...')
## change the location here to zoom to the center
m = folium.Map(location = [42.3756, -93.6397], control_scale = True, zoom_start = 5)

## check if the indexmap geojson files can be rendered properly

folium.GeoJson(data_geojson, 
               tooltip = folium.GeoJsonTooltip(fields=('City', 'County', 'State'),
               aliases=('City', 'County', 'State')),
               show = True).add_to(m)
m